# AutoML project 1



Project team: Oliver Püvi, Li Merila, Karolin Rips, Susanna Metsla, Annika Talvet

In [1]:
#!pip install hyperopt

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import cross_val_score
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings('ignore')

## Reading in the data, preprocessing

In [3]:
df = pd.read_csv('insurance.csv')
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


There are 788 missing values in each column.

In [4]:
df2 = df.dropna()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


Since all the values for those rows are missing, there is no need to impute them; we can simply drop them which is implemented in the code below. We also replace nominal variables (dept and salary) with dummy variables.

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Assuming 'df' is your DataFrame
df = df.dropna()

# Initialize label encoders
label_encoder_dept = LabelEncoder()
label_encoder_salary = LabelEncoder()

# Apply Label Encoding to 'dept' and 'salary' columns
df['sex_encoded'] = label_encoder_dept.fit_transform(df['sex'])
df['smoker_encoded'] = label_encoder_salary.fit_transform(df['smoker'])
df['region_encoded'] = label_encoder_salary.fit_transform(df['region'])
df['charges'] = (df['charges']-df['charges'].min()) /( df['charges'].max() -df['charges'].min())



# Drop original 'dept' and 'salary' columns
encoded_df = df.drop(['sex', 'smoker', 'region'], axis=1)

# Ensure the DataFrame is of type float64
encoded_df = encoded_df.astype('float64')

# Check data types
print(encoded_df.dtypes)


age               float64
bmi               float64
children          float64
charges           float64
sex_encoded       float64
smoker_encoded    float64
region_encoded    float64
dtype: object


In [6]:
encoded_df.describe()

,age,bmi,children,charges,sex_encoded,smoker_encoded,region_encoded
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,0.193916,0.505232,0.204783,1.515695
std,14.049960,6.098187,1.205493,0.193301,0.500160,0.403694,1.104885
min,18.000000,15.960000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,27.000000,26.296250,0.000000,0.057757,0.000000,0.000000,1.000000
50%,39.000000,30.400000,1.000000,0.131849,1.000000,0.000000,2.000000
75%,51.000000,34.693750,2.000000,0.247700,1.000000,0.000000,2.000000
max,64.000000,53.130000,5.000000,1.000000,1.000000,1.000000,3.000000


## Baseline

To construct the baseline, we are trying a set of possible machine learning algorithms (13 algorithms) using their default hyperparamters and we choose the one with the highest performance for comparison (baseline model).

In [7]:
X = encoded_df.drop('charges', axis = 1)
y = encoded_df['charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 43)

seed=43

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(random_state=seed),
    "Lasso": Lasso(random_state=seed),
    "Elastic Net": ElasticNet(random_state=seed),
    "Bayesian Ridge Regression": BayesianRidge(),
    "Decision Tree": DecisionTreeRegressor(random_state=seed),
    "Random Forest": RandomForestRegressor(random_state=seed),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=seed),
    "AdaBoost Regressor": AdaBoostRegressor(random_state=seed),
    "ExtraTrees Regressor": ExtraTreesRegressor(random_state=seed),
    "KNeighbors Regressor": KNeighborsRegressor(),
    "Support Vector Regressor": SVR(),
    "Gaussian Process Regressor": GaussianProcessRegressor(random_state=seed)
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    results[name] = mse

for name, mse in results.items():
    print(f"{name}: MSE = {mse}")

baseline_model_name = min(results, key=results.get)
baseline_mse = results[baseline_model_name]

print(f"\nBaseline Model: {baseline_model_name} with MSE = {baseline_mse}")

Linear Regression: MSE = 0.010967954600655885
Ridge Regression: MSE = 0.010960374581455878
Lasso: MSE = 0.036034381276254246
Elastic Net: MSE = 0.03455210190454312
Bayesian Ridge Regression: MSE = 0.010964964161007174
Decision Tree: MSE = 0.01198289440772746
Random Forest: MSE = 0.008758034538563957
Gradient Boosting Regressor: MSE = 0.007859261892098237
AdaBoost Regressor: MSE = 0.009153800430342463
ExtraTrees Regressor: MSE = 0.00948879524610876
KNeighbors Regressor: MSE = 0.038504238389515055
Support Vector Regressor: MSE = 0.023616965884986924
Gaussian Process Regressor: MSE = 0.04751194054713433

Baseline Model: Gradient Boosting Regressor with MSE = 0.007859261892098237


## Studying the potential pipeline structure - gradient boost

Based on the problem at hand, we study the potential pipeline structure,
algorithms or feature transformers at each step, hyper-parameters ranges. We are using hyperOpt with the potential search space to beat the baseline.

class sklearn.ensemble.RandomForestRegressor(n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=1.0, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)[source]

In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from hyperopt import hp, fmin, rand, Trials, STATUS_OK, space_eval
from sklearn.metrics import mean_squared_error
import numpy as np

# Feature and target separation
X = encoded_df.drop('charges', axis=1)  # Features
y = encoded_df['charges']               # Target

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining a joint search space for hyperparameter optimization
space = {
    'regressor': hp.choice('regressor', [
        {
            'model': GradientBoostingRegressor,
            'n_estimators': hp.choice('n_estimators', [50, 100, 200, 300, 400, 500]),
            'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
            'subsample': hp.uniform('subsample', 0.5, 1.0),
            'max_depth': hp.choice('max_depth', [3, 5, 7, 9, None]),
            'min_samples_split': hp.choice('min_samples_split', [2, 4, 6, 8, 10]),
            'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4, 6, 8]),
            'max_features': hp.choice('max_features', ['sqrt', 'log2', None])
        }
    ]),

    'preprocessor': hp.choice('preprocessor', [
        {'standardscaler': StandardScaler},
        {'minmaxscaler': MinMaxScaler}
    ])
}

# Defining a function to optimize
def objective(params):
    regressor_params = params['regressor']
    preprocessor_params = params['preprocessor']

    preprocessor = list(preprocessor_params.values())[0]()
    regressor_model = regressor_params['model']
    regressor_args = {k: v for k, v in regressor_params.items() if k != 'model'}
    regressor = regressor_model(**regressor_args)

    pipeline = make_pipeline(preprocessor, regressor)
    score = -cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

    return {'loss': score, 'status': STATUS_OK}

# Optimizing hyperparameters
trials = Trials()
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=1500, trials=trials)

# Extracting the best parameters
parameter_values = space_eval(space, best_params)
best_preprocessor = list(parameter_values['preprocessor'].values())[0]
best_regressor = parameter_values['regressor']['model']
remove_model = parameter_values['regressor'].pop('model')
best_param_values = parameter_values['regressor']

# Train the best model on the entire training data
best_pipeline = make_pipeline(best_preprocessor(), best_regressor(**best_param_values))
best_pipeline.fit(X_train, y_train)

# Evaluate the model on the test data
y_pred_test = best_pipeline.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred_test)

# Results
print(f"Best preprocessor: {best_preprocessor.__name__}")
print(f"Best model: {best_regressor.__name__}")
print(f"Best hyperparameters: {best_param_values}")
print(f"Test MSE: {test_mse}")


100%|███| 1500/1500 [08:57<00:00,  2.79trial/s, best loss: 0.005296118354639145]
Best preprocessor: StandardScaler
Best model: GradientBoostingRegressor
Best hyperparameters: {'learning_rate': 0.07702998502400535, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 8, 'min_samples_split': 10, 'n_estimators': 50, 'subsample': 0.6040597570679456}
Test MSE: 0.004710674538073604


In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from hyperopt import hp, fmin, rand, Trials, STATUS_OK, space_eval
from sklearn.metrics import mean_squared_error
import numpy as np

np.random.seed(42)

features = encoded_df.drop('charges', axis=1)
target = encoded_df['charges']

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(features, target, test_size=0.2, random_state=43)

hyperparam_space_rf = {
    'rf_regressor': {
        'model': RandomForestRegressor,
        'n_estimators': hp.choice('rf_n_estimators', [50, 100, 200, 300, 400, 500]),
        'max_depth': hp.choice('rf_max_depth', [5, 10, 15, 20, None]),
        'min_samples_split': hp.choice('rf_min_samples_split', [2, 4, 6, 8, 10]),
        'min_samples_leaf': hp.choice('rf_min_samples_leaf', [1, 2, 4, 6, 8]),
        'max_features': hp.choice('rf_max_features', ['sqrt', 'log2', None]),
        'bootstrap': hp.choice('rf_bootstrap', [True, False])
    },
    'preprocessor': hp.choice('preprocessor_rf', [
        {'standardscaler': StandardScaler},
        {'minmaxscaler': MinMaxScaler}
    ])
}

def objective_rf(params):
    regressor_params_rf = params['rf_regressor'].copy()  # Make a copy of the parameters
    preprocessor_params_rf = params['preprocessor']

    # Remove the 'model' key from the regressor parameters
    regressor_params_rf.pop('model', None)

    preprocessor_rf = list(preprocessor_params_rf.values())[0]()
    rf_model = RandomForestRegressor(**regressor_params_rf)

    pipeline_rf = make_pipeline(preprocessor_rf, rf_model)
    score_rf = -cross_val_score(pipeline_rf, X_train_rf, y_train_rf, cv=5, scoring='neg_root_mean_squared_error').mean()

    return {'loss': score_rf, 'status': STATUS_OK}

trials_rf = Trials()
best_params_rf = fmin(fn=objective_rf, space=hyperparam_space_rf, algo=rand.suggest, max_evals=50, trials=trials_rf)

optimized_params_rf = space_eval(hyperparam_space_rf, best_params_rf)
best_preprocessor_rf = list(optimized_params_rf['preprocessor'].values())[0]
rf_regressor_params = optimized_params_rf['rf_regressor']
rf_regressor_params.pop('model', None)
best_rf_regressor = RandomForestRegressor(**rf_regressor_params)
optimized_pipeline_rf = make_pipeline(best_preprocessor_rf(), best_rf_regressor)
optimized_pipeline_rf.fit(X_train_rf, y_train_rf)
y_pred_rf = optimized_pipeline_rf.predict(X_test_rf)
test_mse_rf = mean_squared_error(y_test_rf, y_pred_rf)

print(f"Best preprocessor: {best_preprocessor_rf.__name__}")
print(f"Best RandomForest model: {best_rf_regressor.__class__.__name__}")
print(f"Optimized hyperparameters: {rf_regressor_params}")
print(f"Test MSE: {test_mse_rf}")


100%|████████| 50/50 [01:03<00:00,  1.28s/trial, best loss: 0.06677529123256261]
Best preprocessor: StandardScaler
Best RandomForest model: RandomForestRegressor
Optimized hyperparameters: {'bootstrap': True, 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 500}
Test MSE: 0.007707672927334427


In [10]:
optimized_params_rf = space_eval(hyperparam_space_rf, best_params_rf)
best_preprocessor_rf = list(optimized_params_rf['preprocessor'].values())[0]

# Remove the 'model' key from the regressor parameters
rf_regressor_params = optimized_params_rf['rf_regressor']
rf_regressor_params.pop('model', None)

best_rf_regressor = RandomForestRegressor(**rf_regressor_params)
optimized_pipeline_rf = make_pipeline(best_preprocessor_rf(), best_rf_regressor)
optimized_pipeline_rf.fit(X_train_rf, y_train_rf)

y_pred_rf = optimized_pipeline_rf.predict(X_test_rf)
test_mse_rf = mean_squared_error(y_test_rf, y_pred_rf)

print(f"Best preprocessor: {best_preprocessor_rf.__name__}")
print(f"Best RandomForest model: {best_rf_regressor.__class__.__name__}")
print(f"Optimized hyperparameters: {rf_regressor_params}")
print(f"Test MSE: {test_mse_rf}")


Best preprocessor: StandardScaler
Best RandomForest model: RandomForestRegressor
Optimized hyperparameters: {'bootstrap': True, 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 500}
Test MSE: 0.007689209812016932


## Monitoring the performance of the constructed pipeline

Monitoring the the performance of the constructed pipeline from the previous step across different time budgets (number of iterations) and reporting the least time budget that you are able to outperform the baseline.

In [11]:

for trial in trials:
  trial_nr = trial['tid']+1
  pipeline_mse_trial = trial['result']['loss']
  if (pipeline_mse_trial < baseline_mse):
    #print(f"Trial {trial_nr}: Pipeline MSE is SMALLER than baseline MSE.")
    
    break
  else:
    #print(f"Trial {trial_nr}: Pipeline MSE is bigger than baseline MSE.")
    print(pipeline_mse_trial, baseline_mse)

0.008455846103965475 0.007859261892098237


## Statistical test

Determining whether the difference in performance between the constructed pipeline and the baseline is statistically significant.